In [1]:
import pandas as pd

df_data_1 = pd.read_csv('bankloanData.csv')
df_data_1.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default,preddef1,preddef2,preddef3
0,41,3,17,12,176,9.3,11.359392,5.008608,1,0.808394,0.788640,0.213043
1,27,1,10,6,31,17.3,1.362202,4.000798,0,0.198297,0.128445,0.436903
2,40,1,15,14,55,5.5,0.856075,2.168925,0,0.010036,0.002987,0.141023
3,41,1,15,14,120,2.9,2.658720,0.821280,0,0.022138,0.010273,0.104422
4,24,2,2,0,28,17.3,1.787436,3.056564,1,0.781588,0.737885,0.436903


In [2]:
df_data_1.dtypes

age           int64
ed            int64
employ        int64
address       int64
income        int64
debtinc     float64
creddebt    float64
othdebt     float64
default      object
preddef1    float64
preddef2    float64
preddef3    float64
dtype: object

In [3]:
df_data_1.values

array([[41, 3, 17, ..., 0.808394327359702, 0.7886404318214371,
        0.21304337612811897],
       [27, 1, 10, ..., 0.19829747615910395, 0.128445387038174,
        0.43690300550604605],
       [40, 1, 15, ..., 0.0100361080990023, 0.00298677834821412,
        0.141022623460993],
       ..., 
       [48, 1, 13, ..., 0.0301374981044824, 0.0325702625943738,
        0.24801041775523303],
       [35, 2, 1, ..., 0.26900345101699397, 0.37854649636973203,
        0.181814378077261],
       [37, 1, 20, ..., 0.006397812918809229, 0.0111731232851226,
        0.30304155578236497]], dtype=object)

In [4]:
import numpy as np

# Make a numpy array from the dataframe, except remove rows with no value for 'default'
i = list(df_data_1.columns.values).index('default')
data = np.array([x for x in df_data_1.values if x[i] in ['0', '1']])

# Remove the columns for preddef1, predef2 and preddef3
data = np.delete(data, slice(9,12), axis=1)

# Separate the 'predictors' (aka 'features') from the dependent variable (aka 'label') that we will learn how to predict
predictors = np.delete(data, 8, axis=1)
dependent = np.delete(data, slice(0, 8), axis=1)

In [5]:
# Convert the label type to numeric categorical representing the classes to predict (binary classfier)
dependent = dependent.astype(int)

# And flatten it to one dimensional for use as the expected output label vector in TensorFlow
dependent = dependent.flatten()
dependent

# Convert all the predictors to float to simplify this demo TensorFlow code
predictors = predictors.astype(float)

# Get the shape of the predictors
m, n = predictors.shape
m, n

(700, 8)

In [6]:
predictors[0]

array([  41.      ,    3.      ,   17.      ,   12.      ,  176.      ,
          9.3     ,   11.359392,    5.008608])

In [7]:
dependent

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1,

In [8]:
# Partition the input data into a training set and a test set

m_train = 500
m_test = m - m_train

predictors_train = predictors[:m_train]
dependent_train = dependent[:m_train]

predictors_test = predictors[m_train:]
dependent_test = dependent[m_train:]

# Gets a batch of the training data. 
# NOTE: Rather than loading a whole large data set as above and then taking array slices as done here, 
#       This method can connect to a data source and select just the batch needed.
def get_training_batch(batch_num, batch_size):
    lower = batch_num * (m_train // batch_size)
    upper = lower + batch_size
    return predictors_train[lower:upper], dependent_train[lower:upper]

In [9]:
import tensorflow as tf

# Make this notebook's output stable across runs
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

# A method to build a new neural net layer of a given size,  
# fully connect it to a given preceding layer X, and 
# compute its output Z either with or without (default) an activation function
# Call with activation=tf.nn.relu or tf.nn.sigmoid or tf.nn.tanh, for examples

def make_nn_layer(layer_name, layer_size, X, activation=None):
    with tf.name_scope(layer_name):
        X_size = int(X.get_shape()[1])
        SD = 2 / np.sqrt(X_size)
        weights = tf.truncated_normal((X_size, layer_size), dtype=tf.float64, stddev=SD)
        W = tf.Variable(weights, name='weights')
        b = tf.Variable(tf.zeros([layer_size], dtype=tf.float64), name='biases')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [10]:
# Make the neural net structure

n_inputs = n
n_hidden1 = n 
### n_hidden2 = n // 2
n_outputs = 2   # Two output classes: defaulting or non-defaulting on loan

X = tf.placeholder(tf.float64, shape=(None, n_inputs), name='X')

with tf.name_scope('nn'):
    hidden1 = make_nn_layer('hidden1', n_hidden1, X, activation=tf.nn.relu)
    hidden2 = hidden1
    ### hidden2 = make_nn_layer('hidden2', n_hidden2, hidden1, activation=tf.nn.relu)
    outputs = make_nn_layer('outputs', n_outputs, hidden2) 
    outputs = tf.identity(outputs, "nn_output")
    
y = tf.placeholder(tf.int64, shape=(None), name='y')

In [11]:
# Define how the neural net will learn

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=outputs)
    loss = tf.reduce_mean(xentropy, name='l')
    
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("test"):
    correct = tf.nn.in_top_k(tf.cast(outputs, tf.float32), y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    

In [12]:
# Set up the ability to save and restore the trained neural net...

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [13]:
# ... and make a subdirectory space in which to save the model files (only need to run this once)
!mkdir "../datasets"
!mkdir "../datasets/Neural Net"

In [14]:
# TRAINING TIME

# This is how many times to use the full set of training data
n_epochs = 3000

# For a larger training set, it's typically necessary to break training into
# batches so only the memory needed to store one batch of training data is used
batch_size = 50

with tf.Session() as training_session:
    init.run()
    
    for epoch in range(n_epochs):
        
        # Shuffling (across batches) is easier to do for small data sets and
        # helps increase accuracy
        training_set = [[pt_elem, dependent_train[i]] for i, pt_elem in enumerate(predictors_train)]
        np.random.shuffle(training_set)
        predictors_train = [ts_elem[0] for ts_elem in training_set]
        dependent_train = [ts_elem[1] for ts_elem in training_set]
        
        # Loop through the whole training set in batches
        for batch_num in range(m_train // batch_size):
            X_batch, y_batch = get_training_batch(batch_num, batch_size)
            training_session.run(training_op, feed_dict={X: X_batch, y: y_batch})

        if epoch % 100 == 99:
            acc_train = accuracy.eval(feed_dict={X: predictors_train, y: dependent_train})
            acc_test = accuracy.eval(feed_dict={X: predictors_test, y: dependent_test})
            print(epoch+1, "Training accuracy:", acc_train, "Testing accuracy:", acc_test)

    save_path = saver.save(training_session, "../datasets/Neural Net/Neural Net.ckpt")
    
    # A quick test with the trained model 
    Z = outputs.eval(feed_dict={X: predictors_test[:20]})
    dependent_pred = np.argmax(Z, axis=1)
    print("")
    print("Actual classes:   ", dependent_test[:20])  
    print("Predicted classes:", dependent_pred)

(100, 'Training accuracy:', 0.75999999, 'Testing accuracy:', 0.755)
(200, 'Training accuracy:', 0.77399999, 'Testing accuracy:', 0.79000002)
(300, 'Training accuracy:', 0.79799998, 'Testing accuracy:', 0.81999999)
(400, 'Training accuracy:', 0.79799998, 'Testing accuracy:', 0.83499998)
(500, 'Training accuracy:', 0.80800003, 'Testing accuracy:', 0.82499999)
(600, 'Training accuracy:', 0.796, 'Testing accuracy:', 0.815)
(700, 'Training accuracy:', 0.80199999, 'Testing accuracy:', 0.81)
(800, 'Training accuracy:', 0.78799999, 'Testing accuracy:', 0.815)
(900, 'Training accuracy:', 0.81, 'Testing accuracy:', 0.815)
(1000, 'Training accuracy:', 0.81999999, 'Testing accuracy:', 0.81)
(1100, 'Training accuracy:', 0.81800002, 'Testing accuracy:', 0.81)
(1200, 'Training accuracy:', 0.80599999, 'Testing accuracy:', 0.80500001)
(1300, 'Training accuracy:', 0.80199999, 'Testing accuracy:', 0.815)
(1400, 'Training accuracy:', 0.81199998, 'Testing accuracy:', 0.81999999)
(1500, 'Training accuracy:'

In [15]:
# Restore the saved model and use it to perform inference on a "received" new set of data

# We will simulate "receiving" the new data by taking a slice of the test set.
predictors_received = predictors_test[20:40]

import tensorflow as tf_inference

with tf_inference.Session() as inference_session:
    inf_saver = tf_inference.train.import_meta_graph('../datasets/Neural Net/Neural Net.ckpt.meta')
    inf_saver.restore(inference_session, tf_inference.train.latest_checkpoint('../datasets/Neural Net/'))
    
    graph = tf_inference.get_default_graph()    
    nn_output = graph.get_tensor_by_name("nn/nn_output:0")

    Z = inference_session.run(nn_output, feed_dict={X: predictors_received})
    dependent_pred = np.argmax(Z, axis=1)
    
    dependent_prob = inference_session.run(tf_inference.nn.softmax(nn_output), feed_dict={X: predictors_received})

    confidences = [p[dependent_pred[i]] for i, p in enumerate(dependent_prob)]
    
print("Actual classes:   ", dependent_test[20:40])
print("Predicted classes:", dependent_pred)
print("")
print("Confidences: ", confidences)
print("")
print("Probabilities: ", dependent_prob)

INFO:tensorflow:Restoring parameters from ../datasets/Neural Net/Neural Net.ckpt
('Actual classes:   ', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1]))
('Predicted classes:', array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1]))

('Confidences: ', [0.72115788841227302, 0.80607047578305713, 0.83934041973914708, 0.99522251895376912, 0.63583707224033514, 0.92301370061024191, 0.9246727235093638, 0.78439759502501416, 0.99999999999583422, 0.64561868763996888, 0.72736408931601459, 0.64561868763996888, 0.83959328566375235, 0.94291801514712859, 0.68170960394665236, 0.99981345698850999, 0.96778595641239296, 0.59248611260528017, 0.99293831799558985, 0.52204835458766652])

('Probabilities: ', array([[  7.21157888e-01,   2.78842112e-01],
       [  8.06070476e-01,   1.93929524e-01],
       [  8.39340420e-01,   1.60659580e-01],
       [  9.95222519e-01,   4.77748105e-03],
       [  6.35837072e-01,   3.64162928e-01],
       [  9.23013701e-01,   7.69862994e-02],
    

In [ ]:
# For when you want to wipe out the training and do it again
!rm -rf "../datasets/Neural Net"

In [16]:
!ls -l "../datasets/Neural Net"

total 80
-rw-r--r--  1 boyerj  staff    720 10 Nov 13:26 Neural Net.ckpt.data-00000-of-00001
-rw-r--r--  1 boyerj  staff    238 10 Nov 13:26 Neural Net.ckpt.index
-rw-r--r--  1 boyerj  staff  25750 10 Nov 13:26 Neural Net.ckpt.meta
-rw-r--r--  1 boyerj  staff     87 10 Nov 13:26 checkpoint


In [ ]:
!ls -l ".."

In [ ]:
!rm -rf "../datasets"